In [1]:
import numpy as np
from liberate import fhe
from liberate.fhe import presets


In [4]:
params = presets.params["gold"]
print(params)
params["devices"] = [0]
# params["scale_bits"] = 49
engine = fhe.ckks_engine(verbose=True, **params)

{'logN': 16, 'num_special_primes': 4, 'devices': [0]}
[2023-10-25 01:20:59.726431] I have received the context:


I have received inputs:
        buffer_bit_length		= 62
        scale_bits			= 40
        logN				= 16
        N				= 65,536
        Number of special primes	= 4
        Number of scales		= 34
        Cache folder			= '/home/kyuyeon/.pyenv/versions/compyler/lib/python3.10/site-packages/liberate/fhe/cache/resources'
        Security bits			= 128
        Quantum security model		= post_quantum
        Security sampling distribution	= uniform
        Number of message bits		= 60
        In total I will be using '1,660' bits out of available maximum '1,661' bits.
        And is it secured?		= True
My RNS primes are [1099510054913, 1099515691009, 1099507695617, 1099516870657, 1099506515969, 1099521458177, 1099503894529, 1099522375681, 1099490000897, 1099523555329, 1099489607681, 1099525128193, 1099486855169, 1099526176769, 1099484889089, 1099529060353, 1099480956929, 109953522073

In [17]:
target_N = 10**7 # 10,000,000
# target_N = 100000
required_num_ct = round(target_N/engine.num_slots)
required_num_ct

# print(required_num_ct)

305

In [14]:
# generate Keys

sk = engine.create_secret_key()
pk = engine.create_public_key(sk)
evk = engine.create_evk(sk)
gk = engine.create_galois_key(sk)

# 02. Mean

In [15]:
# generate Data

cts = []
ms = []
sample_range = 1
for _ in range(required_num_ct):
    m = engine.example(-sample_range, sample_range).real
    ct = engine.encorypt(m, pk, level=engine.num_levels - 4)
    # ct = engine.encorypt(m, pk, level=0)
    ms.append(m)
    cts.append(ct)

In [16]:
# cacle mean
    
result = engine.add(cts[0], cts[1])

for ct in cts[2:]:
    result = engine.add(result, ct)

ct_mean = engine.mean(result, gk, alpha=required_num_ct)

calculated_mean = engine.decrode(ct_mean, sk).mean()
target_mean = np.array(ms).mean()

print("====="*20)
print(f">>\tcalc mean\t: {calculated_mean.real:20.13e}")
print(f">>\ttarget mean\t: {target_mean:20.13e}")
print(f">>\terror\t\t: {(calculated_mean - target_mean).real:20.13e}")

>>	calc mean	: -2.5286213570915e-03
>>	target mean	: -2.5286214383698e-03
>>	error		:  8.1278284016356e-11


# 03. Var(iance)

In [28]:
# generate data

cts = []
ms = []
t = 1

for _ in range(required_num_ct):
    m = engine.example(-t, t).real
#     ct = engine.encorypt(m, pk, level=engine.num_levels - 8)
    ct = engine.encorypt(m, pk, level=0)
    ms.append(m)
    cts.append(ct)

In [30]:
%%time

# calc Var 

result = engine.add(cts[0], cts[1])

for ct in cts[2:]:
    result = engine.add(result, ct)

ct_mean = engine.mean(result, gk, alpha=required_num_ct)

devs = []

# for idx, ct in enumerate(cts):
#     cts[idx] = engine.level_up(ct, ct_mean.level)

for ct in cts:
    dev = engine.sub(ct, ct_mean)

    dev = engine.square(dev, evk, relin=False)
    devs.append(dev)


result = engine.cc_add(devs[0], devs[1])

for dev in devs[2:]:
    result = engine.cc_add(dev, result)

result = engine.relinearize(result, evk)

ct_var = engine.mean(result, gk, alpha=required_num_ct)

calculated_var = engine.decrode(ct_var, sk).mean()
target_var = np.array(ms).real.var() + np.array(ms).imag.var() * 1j

print(calculated_var, target_var, calculated_var - target_var)
print(f">>\tcalc var\t: {calculated_var.real:20.13e}")
print(f">>\ttarget var\t: {target_var.real:20.13e}")
print(f">>\terror\t\t: {(calculated_var - target_var).real:20.13e}")

(0.3333474414574396-1.2862320071882383e-12j) (0.33334693483323596+0j) (5.066242036444457e-07-1.2862320071882383e-12j)
>>	calc var	:  3.3334744145744e-01
>>	target var	:  3.3334693483324e-01
>>	error		:  5.0662420364445e-07
CPU times: user 1.42 s, sys: 65.7 ms, total: 1.48 s
Wall time: 1.76 s


# 04. Standard Deviation(STD)

In [8]:
# generate data

cts = []
ms = []
t = 1

for _ in range(required_num_ct):
    m = engine.example(-t, t).real
#     ct = engine.encorypt(m, pk, level=engine.num_levels - 8)
    ct = engine.encorypt(m, pk, level=0)
    ms.append(m)
    cts.append(ct)

In [9]:
%%time

##############
#### mean ####
##############

result = engine.add(cts[0], cts[1])

for ct in cts[2:]:
    result = engine.add(result, ct)

ct_mean = engine.mean(result, gk, alpha=required_num_ct)

devs = []

#############
#### var ####
#############
# for idx, ct in enumerate(cts):
#     cts[idx] = engine.level_up(ct, ct_mean.level)

for ct in cts:
    dev = engine.sub(ct, ct_mean)

    dev = engine.square(dev, evk, False)
    devs.append(dev)


result = engine.cc_add(devs[0], devs[1])

for dev in devs[2:]:
    result = engine.cc_add(dev, result)

result = engine.relinearize(result, evk)


ct_var = engine.mean(result, gk, alpha=required_num_ct)

calculated_var = engine.decrode(ct_var, sk).mean()
target_var = np.array(ms).real.var() + np.array(ms).imag.var() * 1j

print(calculated_var, target_var, calculated_var - target_var)

(0.3334105539493426+5.144928028752952e-12j) (0.33341004722874884+0j) (5.067205937625552e-07+5.144928028752952e-12j)
CPU times: user 1.39 s, sys: 64.1 ms, total: 1.45 s
Wall time: 2.07 s


In [11]:
#############
#### std ####
#############

ct_std = engine.sqrt(ct_var, evk=evk)

In [27]:
print(f"""{engine.decrode(ct_std, sk)[0].real:10.6f} | {np.std(ms):10.6f}  |  {engine.decrode(ct_std, sk)[0].real-np.std(ms):20.15f}""")

  0.577416 |   0.577417  |    -0.000000478307912


In [31]:
print(f">>\tcalc var\t: {calculated_var.real:20.13e}")
print(f">>\ttarget var\t: {target_var.real:20.13e}")
print(f">>\terror\t\t: {(calculated_var - target_var).real:20.13e}")

>>	calc var	:  3.3334744145744e-01
>>	target var	:  3.3334693483324e-01
>>	error		:  5.0662420364445e-07


In [35]:
calculated_std = engine.decrode(ct_std, sk)
calculated_std[0].real

0.5774162232419724

0.5773620483139119